In [1]:
import pickle as pkl
from tqdm import tqdm
from datetime import datetime
import numpy as np
import pandas as pd
from collections import defaultdict
import math
import torch

In [2]:
def open_file(path):
    with open(path, "rb") as f:
        obj = pkl.load(f)
    f.close()
    return obj

In [3]:
user_cat = open_file("FilterFeatures/user_cat_val.pkl")
item_cat = open_file("FilterFeatures/itens_cat_val.pkl")
index2item = open_file("datasets/out/index2item.pkl")
index2user = open_file("datasets/out/index2user.pkl")

In [30]:
# Quero pegar a sequencia de itens da validação e recomendar os meus itens do treino na mesma sequencia

In [38]:
user_doc = open_file("DataPreProcessing/data_pro/train/User2DocumentMean.pkl")

In [4]:
item_doc = open_file("DataPreProcessing/data_pro/train/Item2Document.pkl")

In [5]:
item2index = open_file("datasets/out/item2index.pkl")
user2index = open_file("datasets/out/user2index.pkl")

In [6]:
user_train = open_file("DataPreProcessing/data_pro/train/UserItemIteractionsTrain.pkl")

In [7]:
# Users da validação
user_val = open_file("DataPreProcessing/data_pro/train/UserItemIteractionsVal.pkl")
users_validation = user_val.keys()

### Itens por Timestamp

In [8]:
item_docs = open_file("datasets/out/finetuning_2.pkl")

In [4]:
times = [datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z").timestamp() for x in list(item_docs['issued'].values)]
item_per_timestamp = {item:timestamp for item,timestamp in list(zip(list(item_docs['page'].values), times))}

In [6]:
save("FilterFeatures/item_timestamp.pkl", item_per_timestamp)

### User por ultimo item consumido

In [77]:
user_data = open_file("datasets/out/user_iter.pkl")

In [78]:
user_train = open_file("DataPreProcessing/data_pro/train/UserItemIteractionsTrain.pkl")

In [79]:
user_last_iter = {user:item[-1] for user,item in user_train.items()}

### User por ultimo timestamp consumido

In [80]:
user_last_timestamp = {user:item_per_timestamp[index2item[item]] for user, item in user_last_iter.items()}

### Modificando USER_CAT para pegar a sequencia de categorias consumidas pelo meu usuário

In [108]:
user_cat_new = {}

for user, cats in user_cat.items():
    while len(cats) < 10:
        cats.extend(cats)
        cats = cats[:10]

    if len(cats) > 10:
        cats = cats[:10]

    user_cat_new[user] = cats

In [82]:
user_last_timestamp = {user:time for user, time in user_last_timestamp.items() if user in users_validation}

### Filtrando itens por recencia

In [109]:
# Crie um dicionário de itens por categoria
items_by_category = defaultdict(list)
for item, category in item_cat.items():
    items_by_category[category].append(item)

for cat, itens in items_by_category.items():
    itens = [x for x in itens if "esid" not in x]
    items_by_category[cat] = itens

In [110]:
# Função para obter o timestamp de um item
def get_item_timestamp(item):
    return item_per_timestamp.get(item, 0)

# Listas ordenadas por itens menos recentes de cada categoria
ordered_categoria_itens = {
    categoria: sorted(itens, key=get_item_timestamp, reverse=True)
    for categoria, itens in items_by_category.items()
}

## Mais recentes e baseados em categoria para cada usuário

In [111]:
# Função para recomendar itens

# Salvo em MostRecentlyCatUsers

def recommend_items(user):
    user_cat_itens = ordered_categoria_itens.copy()

    last_time = user_last_timestamp[user]
    cats = user_cat_new[user]

    rec_itens = []

    for cat in cats:
        if type(cat) == float: continue
        
        item_list = user_cat_itens[cat]

        # Se a lista de itens da categoria estiver vazia, apenas continue
        if len(item_list) == 0: continue

        # Quero apenas os itens que tem o timestamp maior que o ultimo timestamp do meu user
        item_list = [x for x in item_list if item_per_timestamp[x] > last_time]

        if len(item_list) == 0: continue

        item = item_list.pop(0)
        rec_itens.append(item)
        user_cat_itens[cat] = item_list

    return rec_itens



# Recomende itens para cada usuário
recommended_items_by_user = {}

for user in tqdm(user_last_timestamp.keys()):
    recommended_items = recommend_items(user)
    recommended_items_by_user[user] = recommended_items

  0%|          | 0/112184 [00:00<?, ?it/s]

100%|██████████| 112184/112184 [37:36<00:00, 49.71it/s] 


In [98]:
# recommended_items_by_user = open_file("FilterFeatures/MostRecentlyCatUserReverseFt2.pkl")

In [5]:
def save(path, obj):
    with open(path, "wb") as f:
        pkl.dump(obj, f)
        f.close()

In [112]:
save("FilterFeatures/MostRecentlyCatUserReverseFt2_val.pkl", recommended_items_by_user)

In [1]:
user_val = open_file("DataPreProcessing/data_pro/train/UserItemIteractionsVal.pkl")

NameError: name 'open_file' is not defined

In [113]:
for user, itens in recommended_items_by_user.items():
    val_itens = user_val[user]
    val_itens += itens
    recommended_items_by_user[user] = val_itens

## Mais similares

In [29]:
user_doc = open_file("datasets/out/user_finetune_title.pkl")
user_doc.head()

,userId,title,finetune
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,"Você viu? 'Musa das Estradas' faz vídeo de pé,...","[0.8514806032180786, 0.9547808170318604, -0.98..."
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Agosto Lilás: Itapetininga promove palestras d...,"[-0.960038423538208, -0.8187256455421448, 0.29..."
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,"'Estou arrependido. Fui covarde', diz homem su...","[0.6168979406356812, 0.8666552901268005, 0.008..."
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,"VÍDEO: ‘Me ajuda, por favor! Não! Socorro!’, s...","[-0.9817279577255249, -0.9298797249794006, -0...."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Como fazer o empréstimo do Auxílio Brasil?Empr...,"[-0.48284488916397095, 0.47778788208961487, -0..."


In [32]:
user_doc = {user2index[user]:doc for user, doc in list(zip(list(user_doc['userId'].values), list(user_doc['finetune'].values)))}

In [35]:
for user, itens in tqdm(recommended_items_by_user.items()):
    u_doc = torch.Tensor(user_doc[user])

    sims = []

    for item in itens:
        if type(item) == str:
            i_doc = torch.Tensor(item_doc[item2index[item]])
        else:
            i_doc = torch.Tensor(item_doc[item])

        sim = torch.dot(u_doc, i_doc)

        if type(item) == str:
            sims.append((item, sim))
        else:
            sims.append((index2item[item], sim.item()))

    sims = sorted(sims, key=lambda x: x[1], reverse=True)

    recommended_items_by_user[user] = [x[0] for x in sims]
    

100%|██████████| 112184/112184 [00:42<00:00, 2663.25it/s]


## Teste

In [9]:
times = [datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z").timestamp() for x in list(item_docs['issued'].values)]
item_per_timestamp = {item:timestamp for item,timestamp in list(zip(list(item_docs['page'].values), times))}

In [19]:
new_user_rec = {}

for user, itens in user_val.items():
    train_itens = user_train[user]
    # train_itens = sorted(train_itens, key=lambda item: item_per_timestamp[index2item[item]], reverse=True)
    new_user_rec[user] = itens + train_itens
    new_user_rec[user] = new_user_rec[user][:15]

In [28]:
# for user, itens in new_user_rec.items():
#     sorted_itens = sorted(itens, key=lambda item: item_per_timestamp[index2item[item]])
#     new_user_rec[user] = sorted_itens

## Terminar de preencher a lista do usuário pelos topk itens da base

In [47]:
itens_2_count = user_train.values()
itens_counting = {}

for itens in itens_2_count:
    for item in itens:
        if item not in itens_counting.keys():
            itens_counting[item] = 1
        else:
            itens_counting[item] += 1

In [55]:
topk = dict(sorted(itens_counting.items(), key=lambda item: item[1], reverse=True))
topk = topk.keys()

In [20]:
toprecents = dict(sorted(item_per_timestamp.items(), key=lambda item: item[1]))
toprecents = toprecents.keys()
toprecents = [item2index[x] for x in toprecents]

In [13]:
catrec = open_file("FilterFeatures/recently_user_cat_news.pkl")

In [21]:
for user, itens in new_user_rec.items():
    if len(itens) < 15:
        # for k in topk:
        cat_rec = catrec[user]
        for k in cat_rec:
        # for k in toprecents:
            if k not in itens:
                itens.append(item2index[k])

            if len(itens) == 15:
                break

        new_user_rec[user] = itens

In [22]:
out = {
    "userId": [],
    "acessos_futuros": []
}

In [23]:
users = []
access = []

for user, itens in new_user_rec.items():
    for item in itens:
        users.append(index2user[user])

        access.append(index2item[item])


out["userId"] = users
out["acessos_futuros"] = access

In [115]:
users = []
access = []

for user, itens in recommended_items_by_user.items():
    for item in itens:
        users.append(index2user[user])

        if type(item) == str:
            access.append(item)
        else:
            access.append(index2item[item])

out["userId"] = users
out["acessos_futuros"] = access

In [24]:
df = pd.DataFrame(out)
df.to_csv("results2/ValTrainRecents_15.csv", index=False)

## Prog Trash

In [21]:
user_train = {user:itens for user, items in user_train.items() if user in user_val.keys()}

In [26]:
t = user_train[3]

In [27]:
t2 = user_val[3]

In [28]:
t = t + t2

In [31]:
for i, item in enumerate(t):
    if type(item) == int:
        t[i] = index2item[item]

In [32]:
timestamps = open_file("FilterFeatures/")

['35d5bc14-33bc-429c-9d18-1d12e721d3a4',
 'f9d96d01-a23e-4e91-814f-aab763ef7925',
 'a091120b-a832-44d5-94f4-b0225d53f26e',
 '0e99afbb-2452-470f-a76e-011ac2e1554a',
 'cb5cb083-7ac1-4b35-a228-3de83bbab22e',
 'eb23272d-8e6c-479d-b972-eabeb5f6f3dd',
 '40908b83-0102-4e6c-9467-58dd76658be0',
 'c2fe4ec2-0a35-437b-86f3-f1ce1057fa68']

In [35]:
times = [datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z").timestamp() for x in list(item_docs['issued'].values)]
item_per_timestamp = {item:timestamp for item,timestamp in list(zip(list(item_docs['page'].values), times))}

In [36]:
t = [item_per_timestamp[x] for x in t]

In [37]:
t

[1660122741.0,
 1659708864.0,
 1659707371.0,
 1659725023.0,
 1659707517.0,
 1660525594.0,
 1660527969.0,
 1660530097.0]